# Introduce to deep learningin colab environment (runnable)  
Last update: (10/9/2021)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import library


In [ ]:
from numpy.core.numeric import False_
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras.models import Sequential

## Data import

In [ ]:
# Set up your file path 
data_dir = "/content/drive/MyDrive/food"

# Change you batch size here
# In general, batch_size is depends to your GPU/CPU ability, strong GUP may set a larger number of batch size
# From 2,4,8,16,32,64,128,256,512,1023,2048...
batch_size = 32

# Change you imgSize here
# imgSize is depend on model needed. For example, mobienetV3Large accept (224,224,3) shape, so we set 224 here
imgSize = 224

# split you data into 80% (training) and 20% (validations) for 0.2
dataSplitRate = 0.2

# Set train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="training", validation_split=dataSplitRate,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

# Set valid dataset
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="validation", validation_split=dataSplitRate,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

# print the class info with class array
classNum = len(train_ds.class_names)
print(train_ds.class_names)

# plt the graph
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")

plt.show()

## Data augmentation
If you own data numbers is quite small, you may apply some more data from random adjust the attributes of the images.    
See more ...   
https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing

In [ ]:
# method for data transfer, you may adjust here but we stack on this transfer first
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# Data expend for image augmentation, AKA copy and paste your images by times
# you may adjust the expendRound numbers here, 0 means no expend
expendRound = 1
temp_ds = train_ds
for i in range(expendRound):
    train_ds = train_ds.concatenate(temp_ds)

# Apply the data_augmentation to dataset
train_ds = train_ds.map(lambda image,label:(data_augmentation(image),label))

#Resize and / or rescale
Different model may apply to their data structure. For example,MobileNetV3Large accept the color range input [0,255] and (224,224,3) 

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(imgSize, imgSize),
  # In gereral, if the model require [-1,1] or [0,1] input, you may need to able the below line
  # NO need to apply Rescaling if model apply [0,255]
  #layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1) # [-1,1]
  #layers.experimental.preprocessing.Rescaling(1./255) # [0,1]
])

# Apply the resize_and_rescale to dataset
train_ds = train_ds.map(lambda image,label:(resize_and_rescale(image),label))
valid_ds = valid_ds.map(lambda image,label:(resize_and_rescale(image),label))

**Cont' General references of which model need to be rescale or not**  
To specific, 0-255 means the RGB range, not the image size!  (Update 2/9/2021)
 
**[0,255]**  
- EfficientNetB0-7  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/preprocess_input
- MobileNetV3Large / MobileNetV3Small  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v3/preprocess_input  
- ResNet50 / ResNet101 / ResNet152 (RGB to BGR**)   
https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/preprocess_input  
- VGG16 (RGB to BGR**)   
https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input  
- VGG19 (RGB to BGR**)   
https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/preprocess_input


**[0,1]**  
- DenseNet  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet/preprocess_input  


**[-1,1]**  
- MobileNet  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet/preprocess_input    
- MobileNetV2  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/preprocess_input  
- NASNetLarge / NASNetMobile  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/nasnet/preprocess_input  
- InceptionResNetV2  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_resnet_v2/preprocess_input  
- InceptionV3  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input  
- ResNet101V2 / ResNet152V2 / ResNet50V2  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet_v2/preprocess_input  
- Xception  
https://www.tensorflow.org/api_docs/python/tf/keras/applications/xception/preprocess_input  

See more:  

TensorFlow Doc: https://www.tensorflow.org/api_docs/python/tf/keras/applications  
Keras Applications: https://keras.io/api/applications/  
Model rank: https://paperswithcode.com/sota/image-classification-on-imagenet  


## Set up model
you may select a new model here  
In general, if we are using the model to learn out dataSet, we should set the include_top=False to apply our top layers
You may select the model from:   
TF: https://www.tensorflow.org/api_docs/python/tf/keras/applications  
Keras: https://keras.io/api/applications/

## How to select a good moden for your data
In Case, you may see several attributes from keras applications (Parameters, top-1 acc, top-5 acc)  
**Parameters**: In general, bigger params model trains a higher accuracy. But takes more computer resources to train or predict inupt data.  
**Top-1, Top-5** : The accuracy of predicting a game rules call 'imagenet' with 14,197,122 images for classifications 1000 classes.  
Top-1 acc means the first highest possibility label predicts correctly.  
Top-5 means the first top 5 possibility included the actual label.

For general use, mobienetV3Large, efficientNetB1, efficientNetB2 is good enough for training a light weight and decent accuracy model.  
If your GPU is good enough / you have a lot of times, you may select a larger model like EffNetV2-M or EffV1B3 to train your data.  

See more: https://github.com/leondgarse/keras_efficientnet_v2


In [ ]:
baseModel = tf.keras.applications.MobileNetV3Large(input_shape=(imgSize,imgSize,3), include_top=False, weights='imagenet')

17620992/17605208 [==============================] - 0s 0us/step


## Apply our top layer to model


In [ ]:
model = Sequential([
  baseModel,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(classNum, activation=tf.nn.softmax)
])

## fine tune  
You will see that in baseModel, we apply the weights='imagenet'.  
It means that the model have already trained with imagenet game rules with tuned weights. For out own dataset, we need to apply our own dataset rather than original weights.  
Hence, we may lock up the front layer for using original imagenet weights, then set the end layer with learnable weights.    
Lockup layers may depends on your own dataset numbers and characteristics(features).

In [ ]:
baseModel.trainable = True
print("Layers count", len(baseModel.layers))

fine_tune_at = int( len(baseModel.layers) * 0.7)
for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable = False

Layers count 269


## Model training
**epochsRound** means rounds that the model will training   
**base_learning_rate** means the rate to get to the min loss positions  
Usually with 1, 0.1, 0.001, 0.0001, 0.00001


In [ ]:
epochsRound = 10

checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=epochsRound,validation_data=valid_ds, callbacks=[model_checkpoint_callback] )

model.load_weights(checkpoint_filepath)

test_loss, test_acc = model.evaluate(valid_ds, verbose=2)
print(test_acc)

model.save('yourFirstModel.h5')

## See result

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochsRound)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Other Section / How to use your model with sanic

In [ ]:
# Sanic won't work on colab, but flask can run in colab.
# Don't run this on colab environment
from sanic import Sanic
from sanic.response import json, text
from sanic import response
import os
import aiofiles

from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os
import tensorflow as tf

app = Sanic("App Name")
# upload images will be downloaded to your server
app.config["upload"] = 'D:\\gh code\\codeNotes\\mlorDl\\sanicWebServices\\uploads'

imgSize = 224

label = [
'Apple pie','Baby back ribs','Baklava','Beef carpaccio','Beef tartare','Beet salad',
'Beignets','Bibimbap','Bread pudding','Breakfast burrito','Bruschetta','Caesar salad',
'Cannoli','Caprese salad','Carrot cake','Ceviche','Cheesecake','Cheese plate',
'Chicken curry','Chicken quesadilla','Chicken wings','Chocolate cake','Chocolate mousse','Churros',
'Clam chowder','Club sandwich','Crab cakes','Creme brulee','Croque madame',
'Cup cakes','Deviled eggs','Donuts','Dumplings','Edamame','Eggs benedict',
'Escargots','Falafel','Filet mignon','Fish and chips','Foie gras','French fries',
'French onion soup','French toast','Fried calamari','Fried rice','Frozen yogurt',
'Garlic bread','Gnocchi','Greek salad','Grilled cheese sandwich','Grilled salmon',
'Guacamole','Gyoza','Hamburger','Hot and sour soup','Hot dog','Huevos rancheros','Hummus',
'Ice cream','Lasagna','Lobster bisque','Lobster roll sandwich','Macaroni and cheese','Macarons',
'Miso soup','Mussels','Nachos','Omelette','Onion rings','Oysters','Pad thai','Paella','Pancakes','Panna cotta',
'Peking duck','Pho','Pizza','Pork chop','Poutine','Prime rib','Pulled pork sandwich','Ramen','Ravioli','Red velvet cake',
'Risotto','Samosa','Sashimi','Scallops','Seaweed salad','Shrimp and grits','Spaghetti bolognese','Spaghetti carbonara',
'Spring rolls','Steak','Strawberry shortcake','Sushi','Tacos','Takoyaki','Tiramisu','Tuna tartare','Waffles'
]

model = load_model("D:\\efficientnetV1B1Food101.h5") # Model path

if not os.path.exists(app.config["upload"]):
    os.makedirs(app.config["upload"])

@app.get("/")
async def test(request):
    return json({"hello": "Ha"})

def getresult(imgName):
    imgPath = os.path.join(app.config["upload"], imgName)

    image = Image.open(imgPath)
    image = ImageOps.fit(image, (imgSize, imgSize) , Image.ANTIALIAS)

    data = np.ndarray(shape=(1, imgSize, imgSize, 3), dtype=np.float32)
    data[0] = (np.asarray(image).astype(np.float32) / 127.0) - 1 # Mobienetv2 rescale needs to be / 127.0 and -1 => outcome should be between [-1 to 1]
    #data[0] = np.asarray(image).astype(np.float32)  # Model no need to be rescale => outcome should be between [0 to 255]

    prediction = model.predict(data)
    print(label[np.argmax(prediction)])
    return prediction

# The upload route with POST method
@app.route("/upload", methods=['POST'])
async def getFiles(request):
    fileName = request.files["file"][0].name

    async with aiofiles.open(app.config["upload"] + "/" + fileName, 'wb' ) as f:
        await f.write(request.files["file"][0].body)
    f.close()

    prediction = getresult(fileName)
    return text(label[np.argmax(prediction)])


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)

